In [1]:
import pandas as pd, numpy as np
import glob
import os
from datetime import datetime, timedelta

In [6]:
# In this example we are using mock data, but this code was originally designed to automate this task in real life. 
# This code is responsible for cutting ~3h to 4h of manual work each month to get the banking info on NetSuite.
# Variable descriptions
    # extrato = bank statement
    # template = CSV file frame that should be interpreted by NetSuite
    # contas = dict containing accounting classification of transactions, will be used later with map

cols1 = ['amount', 'created','description', 'fee']

extrato = pd.read_excel('Extrato.xlsx', usecols=cols1)

template = pd.read_csv(fr'Template.csv', sep=';')

contas = {'Customer':'00000010', 'Refund':'00000000', 'Bank2':'00000002', 'Daily Fee': '00000005', 'Bank3':'00000003',
          'Bank4':'00000004'}

extrato

,amount,created,description,fee
0,45.40,2022-08-31T03:06:35.122685+00:00,Customer,0.50
1,42.35,2022-08-31T03:07:19.102177+00:00,Customer,0.50
2,86.44,2022-08-31T03:12:25.016023+00:00,Customer,0.50
3,37.49,2022-08-31T03:14:30.991608+00:00,Customer,0.50
4,38.29,2022-08-31T03:24:36.822175+00:00,Customer,0.50
...,...,...,...,...
6716,47.28,2022-09-02T23:55:33.715732+00:00,Customer,0.28
6717,74.73,2022-09-02T23:57:01.563650+00:00,Customer,0.28
6718,31.86,2022-09-02T23:58:25.355250+00:00,Customer,0.28
6719,48.73,2022-09-02T23:58:58.771998+00:00,Customer,0.28


In [7]:
# Here we have a quick manipulation of the bank statement, because the datetime comes as '2022-mm-ddThour:minute:second'
# so we need to do a str slice to get the correct date and work with a list to get all the dates we will write the CSV file.
extrato['created'] = extrato['created'].str.strip().str[:10]
extrato['amount'].round(2)
extrato['fee'].round(2)
extrato.rename(columns={"created": "date"}, inplace=True)

extrato

,amount,date,description,fee
0,45.40,2022-08-31,Customer,0.50
1,42.35,2022-08-31,Customer,0.50
2,86.44,2022-08-31,Customer,0.50
3,37.49,2022-08-31,Customer,0.50
4,38.29,2022-08-31,Customer,0.50
...,...,...,...,...
6716,47.28,2022-09-02,Customer,0.28
6717,74.73,2022-09-02,Customer,0.28
6718,31.86,2022-09-02,Customer,0.28
6719,48.73,2022-09-02,Customer,0.28


In [8]:
# Storing date as list and removing duplicates to loop. The statement of this bank should always be downloaded with -1 day from
# the day you actually want to analyze. Don't know why it works like that, but we need to remove always the first day of
# the downloaded file to get the exact days we want.
datas = extrato['date'].tolist()
datas2 = np.unique(datas).tolist()
del datas2[0]

datas2

['2022-09-01', '2022-09-02']

In [9]:
# We can do with a simple loop, since the data manipulation is quite simple, actually. We'll look step by step.

for data in datas2:
    
    #String slicing to change the date from AAAA-MM-DD to DD/MM/AAAA   
    data = str(data)
    data2 = data[8:] + '/' + data[5:7] + '/' + data[:4]
    
    # Here we are filtering the positive and negative amounts so we can separate as credit or debit entries, then we concatenate
    # to have a single frame which later will be copied to swap credit and debit and have the accounting counterparts.
    filtro_dia = extrato.loc[extrato['date'] == f'{data}']
    filtro_pos = filtro_dia.loc[filtro_dia['amount'] > 0]
    filtro_neg = filtro_dia.loc[filtro_dia['amount'] < 0]
    
    temp_df = pd.concat([filtro_pos, filtro_neg])
    
    # Simple date manipulation inside the df
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['date'] = temp_df['date'].dt.strftime('%d/%m/%Y')
    
    # Creating the excel file in the template format with the bank infos. 
    template1 = pd.DataFrame({'ExternalID': f'BANK{data}001', 'Data':temp_df['date'], 'Histórico':temp_df['description'], 
                              'Conta':'00000001', 'Crédito':filtro_neg['amount'], 'Débito':filtro_pos['amount'], 
                              'Area':'Product', 'Reporting':'General', 'Product':'General', 'Country':'Brazil',
                              'Project':'General', 'Nome Cliente Fornecedor':''})
    
    # Here we are inserting a row to recognize the daily fee from all transactions.
    tarifa = pd.DataFrame({'ExternalID':f'BANK{data}001', 'Data': data2, 'Histórico':'Bank 001 Daily Fee', 'Conta':'00000001',
                           'Crédito': filtro_dia['fee'].sum(), 'Area':'Finance','Reporting':'Cost to Serve',
                           'Country':'Brazil', 'Project':'General', 'Product': 'General', 'Nome Cliente Fornecedor': ''},
                          index=[0])
    template1 = template1.append(tarifa, ignore_index=True)
    template1
    
    
    # As mentioned before, to get the accountig counterpart, we simply make a copy and swap the credit and debit for those are
    # the same entries.
    template2 = template1.copy()
    template2['Crédito'] = template1['Débito']
    template2['Débito'] = template1['Crédito']
    
    # Here we do the partial map with "contas" dict to get the correct classfication to the counterparts.
    pat = r'\b(?:{})\b'.format('|'.join(contas.keys()))
    
    # This part is the most tricky one. Since we don't have many different entries, we manage to classify all the client entries
    # by exclusion. It's not possible to do this directly because client info comes with name + BR id number only.
    # In this example, we're using mock data with a standard string for customers, but in prod this works as partial map.
    template2['Conta'] = template2['Histórico'].str.extract('('+pat+')',expand=False).map(contas).fillna('00000010')
    
    # Concatenating the two frames to get the final result.
    template1 = pd.concat([template1, template2])
    template1.reset_index(drop=True, inplace=True)
    template1
    
    # Some quick str manipulation to get the data in a format that NetSuite can read perfectly. Probably will change
    # this to a more clean coding when I have some spare time, haha!
    template1['Crédito'] = template1['Crédito'].astype(str)
    template1['Débito'] = template1['Débito'].astype(str)

    template1['Crédito'] = template1['Crédito'].str.replace('-','')
    template1['Crédito'] = template1['Crédito'].str.replace('.',',')

    template1['Débito'] = template1['Débito'].str.replace('-','')
    template1['Débito'] = template1['Débito'].str.replace('.',',')
    template1['ExternalID'] = template1['ExternalID'].str.replace('/','')
    template1['Data'] = template1['Data'].str.replace('-','/')
    
    # The thing with copying df is that it returns NaN values as "nan" string, so we need to replace to the Nan values again.
    template1 = template1.replace('nan', np.nan)
    
    # Manipulation completed, so we can generate the csv file ready to input on NetSuite.
    template1.to_csv(fr'Bank 001 Statement{data}.csv', index=False, encoding='utf-8-sig')
    

C:\Users\gporto\AppData\Local\Temp\ipykernel_24444\3380526322.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  template1 = template1.append(tarifa, ignore_index=True)
C:\Users\gporto\AppData\Local\Temp\ipykernel_24444\3380526322.py:62: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  template1['Crédito'] = template1['Crédito'].str.replace('.',',')
C:\Users\gporto\AppData\Local\Temp\ipykernel_24444\3380526322.py:65: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  template1['Débito'] = template1['Débito'].str.replace('.',',')
C:\Users\gporto\AppData\Local\Temp\ipykernel_24444\3380526322.py:3

In [ ]:
# This is a simple matrix return to check all the files generated in just one, making double-checking a lot easier if needed.
matriz = pd.concat(map(pd.read_csv, glob.glob('Extrato Stark SEC 2022*.csv')))
matriz.to_excel('Matriz Extratos Stark SEC.xlsx', index=False)